In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import imageio
import subprocess
import os
from modified_seismic_CPML_2D_pressure_second_order_functions import *

In [3]:
# Directory to save the images
output_dir = 'results'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Compile the Fortran file
compile_command = ["gfortran", "-o", "seismic_pressure_model", "seismic_CPML_2D_pressure_second_order.f90"]

try:
    print("Compiling the Fortran file...")
    subprocess.run(compile_command, check=True)
    print("Compilation successful!")
except subprocess.CalledProcessError as e:
    print(f"Compilation failed: {e}")
    # Exit early if compilation fails
    raise SystemExit("Exiting due to compilation error.")

# Run the compiled executable
run_command = ["./seismic_pressure_model"]

try:
    print("Running the executable...")
    result = subprocess.run(run_command, check=True, capture_output=True, text=True)
    print("Execution successful!\n")
    print(result.stdout)
except subprocess.CalledProcessError as e:
    print(f"Execution failed: {e}")
    print(e.stderr)


Compiling the Fortran file...
Compilation successful!
Running the executable...
Execution successful!

 JSOURCE =           10

 2D acoustic finite-difference code in pressure formulation with C-PML


 NX =          300
 NY =          300

 size of the model along X =    1495.0000000000000     
 size of the model along Y =    1495.0000000000000     

 Total number of grid points =        90000

 d0_x =    1381.5510557964271     
 d0_y =    1381.5510557964271     

 There are            1  receivers

 receiver            1  x_target,y_target =    800.00000000000000        750.00000000000000     
 closest grid point found at distance    0.0000000000000000       in i,j =          161         151

 Courant number is    7.0710676332352770E-002

 Time step #            5  out of         8001
 Time:    3.99999990E-04  seconds
 Max absolute value of pressure =    2.2811259301688223     

 Time step #          125  out of         8001
 Time:    1.23999994E-02  seconds
 Max absolute value of pre

Time: 17 s

In [4]:
# Ruta al archivo de datos
file_path = 'results/kappa_unrelaxed.txt'

# Leer los datos del archivo
data = np.loadtxt(file_path)

# Extraer los índices y valores
i_indices = data[:, 0].astype(int)
j_indices = data[:, 1].astype(int)
values = data[:, 2]

# Determinar el tamaño de la matriz
NX = int(np.max(i_indices))
NY = int(np.max(j_indices))

# Crear la matriz de valores
kappa_matrix = np.zeros((NX, NY))

# Rellenar la matriz con los datos
for i, j, value in zip(i_indices, j_indices, values):
    kappa_matrix[i-1, j-1] = value  # Restar 1 si los índices en el archivo son basados en 1


In [5]:
 

# Cargar los datos del archivo .txt
its = [1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000, 5500, 6000, 6500, 7000, 8000]

# Inicializar los valores globales de vmin y vmax
global_vmin = float('inf')
global_vmax = float('-inf')

# Calcular los valores globales de vmin y vmax
for it in its:
    data = np.loadtxt(f'{output_dir}/pressure_future_{it}.txt')
    pressure = data[:, 2]
    global_vmin = min(global_vmin, np.min(pressure))
    global_vmax = max(global_vmax, np.max(pressure))

# Save the images
for i, it in enumerate(its):
    # Cargar los datos correspondientes
    data = np.loadtxt(f'{output_dir}/pressure_future_{it}.txt')
    x = data[:, 0]
    y = data[:, 1]
    pressure = data[:, 2]

    # Reorganizar los valores de presión en una matriz
    wavefields = pressure.reshape((len(np.unique(y)), len(np.unique(x))))

    # Definir los límites de los ejes
    x_min, x_max = np.min(x), np.max(x)
    y_min, y_max = np.min(y), np.max(y)

    plt.figure()
    plt.title(f'Time: {0.0001*it:.2f} s', fontsize=10)
    plt.xlabel('Distance (m)', fontsize=10)
    plt.ylabel('Depth (m)', fontsize=10)

    # Mostrar el mapa de kappa en gris transparente
    normalized_kappa = kappa_matrix / np.max(kappa_matrix)
    plt.imshow(normalized_kappa, extent=[x_min, x_max, y_min, y_max], aspect=1, cmap='gist_yarg', alpha=0.5, vmin=0, vmax=1)

    # Mostrar el campo de presión sobre el mapa de kappa
    im = plt.imshow(wavefields, extent=[x_min, x_max, y_min, y_max], aspect=1, cmap='seismic', alpha=0.8, vmin=-global_vmax, vmax=global_vmax)

    # Añadir la barra de color con los valores globales
    cbar = plt.colorbar(im)
    cbar.set_label('Pressure')

    # Guardar la figura individualmente con un nombre único
    plt.savefig(f'{output_dir}/wavefields_{it}.png')
    plt.close()

# Create the GIF
images = []
for it in its:
    images.append(imageio.imread(f'{output_dir}/wavefields_{it}.png'))

# Set the frames per second (fps) to control the speed of the GIF
imageio.mimsave(f'pressure_evolution.gif', images, fps=2)

/tmp/ipykernel_25218/1609061751.py:53: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(f'{output_dir}/wavefields_{it}.png'))
